In [16]:
import requests
import json
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from pypfopt import EfficientFrontier # nb pypfopt is shorthand for the package pyportfolioopt
from pypfopt import risk_models
from pypfopt import expected_returns

In [2]:
def date_conv(date):
    return datetime.strptime(date, '%Y-%m-%d')

In [3]:
histories_df= pd.read_csv('histories.csv')
histories_df['date'] = list(map(date_conv,histories_df['date']))
histories_df = histories_df.set_index('date')

In [4]:
def ids_with_histories(histories_df, start_date, end_date):
  investment_df = histories_df[start_date:end_date]
  investment_df.dropna(axis=1, inplace=True) # drop cols with any NaN values
  return investment_df.columns

def uniform_weights_dict(ids_with_histories):
  weight = 1/len(ids_with_histories)
  uniform_weights_dict = {}
  for id in ids_with_histories:
    uniform_weights_dict[id] = weight
  return uniform_weights_dict


def markowitz_weights_dict(histories_df,start_port_date,ids_with_histories, analysis_days=365):
  start_analysis_date = start_port_date - timedelta(analysis_days)
  analysis_df = histories_df[start_analysis_date:start_port_date][ids_with_histories]

  # Calculate expected returns and sample covariance
  mu = expected_returns.mean_historical_return(analysis_df)
  S = risk_models.sample_cov(analysis_df)
  # Optimize for maximal Sharpe ratio
  attempts=0
  while attempts < 10:
    try:
      ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))
      ef.max_sharpe()
      break
    except Exception as e:
      attempts += 1
  try:
    cleaned_weights = ef.clean_weights()
  except Exception as e:
    print("Could not find optimal solution, try changing optimisation constraints or investment set")
  return cleaned_weights


def gen_port_rtns(rebased_df, weights_dict):
  return rebased_df[list(weights_dict.keys())].dot(list(weights_dict.values()))

def gen_rebased_df(histories_df, ids_with_histories, start_date, end_date):
  returns_df = histories_df[ids_with_histories].pct_change(1)
  returns_df[start_date:start_date]=0
  return (1 + returns_df[start_date:end_date]).cumprod()

In [5]:
lookback_years = 5
start_date = date.today() - timedelta(365)
end_date = date.today()
ids_with_histories = ids_with_histories(histories_df,
  start_date, end_date)

/var/folders/ff/pmf9d5156jz_pr_s8ybs3x780000gn/T/ipykernel_63357/2151836904.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  investment_df.dropna(axis=1, inplace=True) # drop cols with any NaN values


In [6]:
uniform_weights_dict = uniform_weights_dict(ids_with_histories[:10])
markowitz_weights_dict = markowitz_weights_dict(histories_df,
  start_date ,ids_with_histories[:10], analysis_days=365)
rebased_df = gen_rebased_df(histories_df, ids_with_histories,
  start_date, end_date)

In [9]:
def gen_all_returns(rebased_df, ids_with_histories,uniform_weights_dict,
  markowitz_weights_dict):
  '''
  A function to generate returns for all portfolios and all coins with full
  histories over the backtest period, rebased to the start of the backtest
  period.
  '''
  uniform_returns = gen_port_rtns(rebased_df, uniform_weights_dict)
  uniform_returns.name = "Uniform"
  markowitz_returns = gen_port_rtns(rebased_df, markowitz_weights_dict)
  markowitz_returns.name = "Markowitz"
  port_returns = uniform_returns.to_frame().join(markowitz_returns)
  return port_returns.join(rebased_df[ids_with_histories])

In [10]:
all_returns_df = gen_all_returns(rebased_df, ids_with_histories,uniform_weights_dict,
  markowitz_weights_dict)

In [11]:
all_returns_df.head()

,Uniform,Markowitz,bitcoin,ethereum,tether,usd-coin,binance-coin,xrp,binance-usd,cardano,...,arweave,compound,kava,holo,gatetoken,fei-protocol,kyber-network,qtum,bancor,1inch
date,,,,,,,,,,,,,,,,,,,,,
2021-05-24,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2021-05-25,1.057165,1.015792,1.035396,1.104084,1.004346,1.008332,1.117736,1.141806,1.004241,1.067167,...,1.015432,1.051228,1.084118,1.231873,1.106947,1.004524,1.037904,1.264676,1.078122,1.064602
2021-05-26,1.103699,1.018414,1.059262,1.182178,0.998764,0.998087,1.211152,1.172456,0.998892,1.180429,...,1.157293,1.142980,1.216742,1.508202,1.167414,1.001874,1.153683,1.288506,1.173015,1.220068
2021-05-27,1.094057,1.016329,1.047055,1.171337,0.998721,0.997785,1.209819,1.151604,0.998899,1.166563,...,1.159935,1.114319,1.253248,1.432746,1.155625,0.999451,1.178437,1.424998,1.192224,1.226486
2021-05-28,1.035221,1.005436,0.988150,1.081310,0.998400,0.997946,1.118547,1.055065,0.998932,1.070248,...,1.011761,1.007871,1.218521,1.269149,1.053198,0.994776,1.072810,1.336266,1.095105,1.125980


In [12]:
def absolute_return(prices):
  'a function to calculate the absolute return given a daily price series'
  abs_rtn = ((prices.iloc[-1]/prices[0])-1)
  return abs_rtn

def annual_return(prices):
  'a function to calculate the annualised return given a daily price series'
  abs_rtn = absolute_return(prices)
  annual_rnt = (pow((abs_rtn/100)+1, 365/len(prices))-1)*100
  return annual_rnt

def max_drawdown(prices):
  '''
  A function to calculate the max drawdown for a given price series "prices"
  as well as the index of the start of the max drawdown period, "start_idx"
  and the index of end of the max drawdwon period, "end index"
  '''
  if type(prices)==type(pd.Series(dtype='object')):
      prices = prices.values
  end_idx = np.argmax(np.maximum.accumulate(prices) - prices) # end of the period
  start_idx = np.argmax(prices[:end_idx]) # start of period
  max_dd = (prices[start_idx]-prices[end_idx])/prices[start_idx]
  return max_dd, start_idx, end_idx

def annual_vol(prices):
  '''
  A function to calculate the annuaised volatility of a price series assuming
  cryptos trade 365 days a year
  '''
  return prices.pct_change().std()*(365**0.5)

In [26]:
%%time 

url = "https://api.coincap.io/v2/assets"

# N.B. here adampt the params dict to only request what you need
payload={'limit': '100'}
headers = {}

response_assets = requests.request("GET", url, params=payload, headers=headers)
assets_json = response_assets.json()

CPU times: user 38.2 ms, sys: 22.1 ms, total: 60.3 ms
Wall time: 677 ms


In [51]:
market_cap_dict = {}
for asset_dict in assets_json['data']:
    market_cap_dict[asset_dict['id']] = int(float(asset_dict['marketCapUsd']))

In [53]:
assets = all_returns_df.columns
performance_df = pd.DataFrame(index = assets)
performance_df['Type'] = ["Portfolio" if x in ['Uniform','Markowitz'] else "Coin" for x in assets]
abs_return = all_returns_df.apply(absolute_return)
ann_vol = all_returns_df.apply(annual_vol)
drawdown_triples = all_returns_df.apply(max_drawdown)
sharpe = abs_return.divide(ann_vol)
market_caps=[]
for asset in assets:
    try:
        market_caps.append(int(market_cap_dict[asset]))
    except:
        market_caps.append(0)
performance_df['Risk adjusted return'] = sharpe *100
performance_df['Return over period'] = abs_return * 100
performance_df['Annual volatility'] = ann_vol *100
performance_df['Max loss'] = drawdown_triples.iloc[0] *100
performance_df['Market cap'] = market_caps

In [55]:
performance_df.round(2).head()
                                 

,Type,Risk adjusted return,Return over period,Annual volatility,Max loss,Market cap
Uniform,Portfolio,-22.24,-12.20,54.84,62.39,0
Markowitz,Portfolio,-32.44,-4.10,12.65,20.49,0
bitcoin,Coin,-32.53,-18.42,56.61,57.98,564237420636
ethereum,Coin,-18.21,-12.59,69.11,59.35,238817158476
tether,Coin,-20.42,-0.35,1.74,1.38,73268815333


In [48]:
market_caps=[]
for asset in assets:
    try:
        market_caps.append(int(market_cap_dict[asset]))
    except:
        market_caps.append(0)

In [49]:
market_caps

[0,
 0,
 564237420636,
 238817158476,
 73268815333,
 53460870289,
 55171443979,
 18541264647,
 18346368674,
 17515646178,
 16745290532,
 11001917114,
 11234130771,
 7787730811,
 8166552066,
 7635320776,
 6577281143,
 5233052871,
 4926418651,
 4791074039,
 4667054135,
 4139464376,
 4059469681,
 3704114916,
 4022432644,
 3351886797,
 3288087835,
 2815947242,
 3539411572,
 2818665288,
 3269122575,
 2723567310,
 1973179577,
 1897055621,
 2022613776,
 1773542413,
 1858408120,
 1582876355,
 1432365865,
 1285088163,
 1459289330,
 1249176930,
 911946815,
 1292895299,
 1327070869,
 760329876,
 1258077538,
 1232920752,
 1444758762,
 1300291461,
 1062463624,
 1083212513,
 993424523,
 1011879265,
 934002994,
 907158090,
 1065469367,
 818145448,
 709251918,
 719142292,
 764685313,
 810068068,
 620807289,
 688840617,
 659905217,
 669737601,
 743232286,
 601257899,
 623988968,
 581409322,
 585301744,
 542798991,
 574126995,
 516407831,
 565880049,
 523461128,
 484782283,
 485835815,
 497033481,
 5715